# Clustering Crypto

In [5]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [85]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv", index_col=0)
crypto_df.head(5)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [86]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df.head(5)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [87]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df[crypto_df["Algorithm"] != ""]

In [88]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=['IsTrading'], inplace=True)
crypto_df.head(5)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [89]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [90]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] != 0]
crypto_df.head(5)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [91]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_df_names = crypto_df[['CoinName']]
crypto_df_names.head(5)

,CoinName
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin


In [92]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=['CoinName'], inplace=True)
crypto_df.head(5)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000


In [93]:
# Use get_dummies() to create variables for text features.
x = pd.get_dummies(crypto_df, columns =["Algorithm", "ProofType"])
x.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
# Standardize the data with StandardScaler().
x_scaled = StandardScaler().fit_transform(x)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [95]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
x_pca = pca.fit_transform(x_scaled)

In [101]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(data=x_pca, columns=['pc1', 'pc2', 'pc3'], index=crypto_df.index)
pca_df

,pc1,pc2,pc3
42,-0.219801,1.209318,-0.361017
404,-0.205192,1.207572,-0.361149
808,-0.181661,0.831770,-0.275172
1337,0.322068,1.992633,-0.426348
BTC,-0.253594,-1.455741,0.102299
...,...,...,...
ZEPH,3.926006,-0.085328,0.138008
GAP,-0.219595,1.209294,-0.361019
BDX,-0.144644,-2.297395,0.242945
ZEN,-0.236586,-1.992289,0.238886


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [97]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)
    
# Create the Elbow Curve using hvPlot
elbow_data = {"k":k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [99]:
# Initialize the K-Means model.
# YOUR CODE HERE
model = KMeans(n_clusters=4)

# Fit the model
# YOUR CODE HERE
model.fit(pca_df)

# Predict clusters
# YOUR CODE HERE
predictions = model.predict(pca_df)

In [100]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pca_df], axis=1, join='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_df_names

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(685, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc1,pc2,pc3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.219801,1.209318,-0.361017,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.205192,1.207572,-0.361149,404Coin,0
808,SHA-256,PoW/PoS,0.000000e+00,0,-0.181661,0.831770,-0.275172,808,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.322068,1.992633,-0.426348,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.253594,-1.455741,0.102299,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.300915,-2.041779,0.180278,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.291109,-1.078267,0.016448,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.204036,1.474026,-0.459548,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.289062,-2.224293,0.321333,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.300836,-2.041788,0.180278,Ethereum Classic,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
